In [27]:
#pip install weaviate-client==3.19.2
#!pip install numpy
#!pip install pandas
#!pip install -U torch==1.12
#!pip install transformers

In [28]:
import argparse
import os
import pathlib
import time
from typing import Any, Dict, List

import numpy as np
import pandas as pd
import torch
import transformers
import weaviate

In [29]:
client = weaviate.Client(
        embedded_options=weaviate.EmbeddedOptions(
            persistence_data_path="/home/l40s/dt-rag/telco/data/output/",
            port=6682,
        )
   )

Binary /root/.cache/weaviate-embedded did not exist. Downloading binary from https://github.com/weaviate/weaviate/releases/download/v1.19.3/weaviate-v1.19.3-linux-amd64.tar.gz
Started /root/.cache/weaviate-embedded: process ID 1380567


{"action":"startup","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2023-12-05T19:59:12-06:00"}
{"action":"startup","auto_schema_enabled":true,"level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2023-12-05T19:59:12-06:00"}
{"level":"warning","msg":"Multiple vector spaces are present, GraphQL Explore and REST API list objects endpoint module include params has been disabled as a result.","time":"2023-12-05T19:59:12-06:00"}
{"action":"grpc_startup","level":"info","msg":"grpc server listening at [::]:50051","time":"2023-12-05T19:59:12-06:00"}
{"action":"restapi_management","level":"info","msg":"Serving weaviate at http://127.0.0.1:6682","time":"2023-12-05T19:59:12-06:00"}
/root/miniconda3/envs/weaviate/lib/python3.10/subprocess.py:1072: ResourceWarning: subprocess 1380567 is still running
  _warn(

In [31]:
!ps -aux | grep weaviate

root     1380212  4.1  0.1 5726836 399408 ?      Sl   19:52   0:19 /root/miniconda3/envs/weaviate/bin/python -m ipykernel_launcher --f=/root/.local/share/jupyter/runtime/kernel-v2-1377957U4LUPEqadhwm.json
root     1380567  1.1  0.0 820944 55484 ?        Sl   19:59   0:00 /root/.cache/weaviate-embedded/weaviate-v1.19.3-8533430c0639ff6721fd01b55d44da58850a60b20c9dd25db1908624d9dca67d --host 127.0.0.1 --port 6682 --scheme http
root     1380633  0.0  0.0   7368  3384 pts/7    Ss+  19:59   0:00 /bin/bash -c ps -aux | grep weaviate
root     1380635  0.0  0.0   6608  2284 pts/7    S+   19:59   0:00 grep weaviate


In [30]:

def main() -> None:
    os.makedirs(args.output, exist_ok=True)
    client = weaviate.Client(
        embedded_options=weaviate.EmbeddedOptions(
            persistence_data_path=args.output,
            port=6682,
        )
    )
    print("Loading model for creating index")
    tokenizer = transformers.AutoTokenizer.from_pretrained(
        args.tokenizer, cache_dir=args.hf_cache_dir
    )
    tokenizer.pad_token = tokenizer.eos_token
    model = transformers.AutoModelForCausalLM.from_pretrained(
        args.embedding_model,
        trust_remote_code=True,
        torch_dtype=torch.bfloat16,
        low_cpu_mem_usage=True,
        cache_dir=args.hf_cache_dir,
    ).to("cuda:0")
    model.eval()
    chunks = pd.concat([pd.read_csv(x) for x in args.chunks_file]).fillna("")
    embeddings = TOKENIZE_LOGIC_MAP[args.tokenize_logic](tokenizer, model, args.prompt, args.prompt)
    print("Creating embedding index")
    start_time = time.time()
    create_weaviate_database(client, chunks.to_dict(orient="records"), embeddings)
    end_time = time.time()
    print(f"Finished creating embedding index, total time {end_time - start_time}")


with torch.inference_mode():
    main()


root     1380212  4.2  0.1 5726656 399328 ?      Sl   19:52   0:19 /root/miniconda3/envs/weaviate/bin/python -m ipykernel_launcher --f=/root/.local/share/jupyter/runtime/kernel-v2-1377957U4LUPEqadhwm.json
root     1380567  1.4  0.0 820944 55484 ?        Sl   19:59   0:00 /root/.cache/weaviate-embedded/weaviate-v1.19.3-8533430c0639ff6721fd01b55d44da58850a60b20c9dd25db1908624d9dca67d --host 127.0.0.1 --port 6682 --scheme http
root     1380629  0.0  0.0   7368  3392 pts/7    Ss+  19:59   0:00 /bin/bash -c ps -aux | grep weaviate
root     1380631  0.0  0.0   6608  2204 pts/7    S+   19:59   0:00 grep weaviate


AttributeError: module 'torch' has no attribute 'inference_mode'